## Imports

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.io import loadmat
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

%matplotlib inline

Set seeds for reproducability

In [2]:
np.random.seed(0)
torch.manual_seed(0)

## Create Datasets

Specify paths to the training data (104x104 pixel images only)

In [3]:
# For working on Martinsried machine
dspl_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/dspl'
dsplRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/dsplRadial'
trac_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/trac'
tracRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/tracRadial'

In [4]:
# For working remotely on CIP machines
# dspl_path = '/home/r/richard/Downloads/train/trainData104/dspl'
# dsplRadial_path = '/home/r/richard/Downloads/train/trainData104/dsplRadial'
# trac_path = '/home/r/richard/Downloads/train/trainData104/trac'
# tracRadial_path = '/home/r/richard/Downloads/train/trainData104/tracRadial'

Load data as `Numpy` arrays first

In [5]:
def data_to_npArrays(dspl_path, dsplRadial_path, trac_path, tracRadial_path):
    number_samples = len([name for name in os.listdir(dspl_path) if os.path.isfile(os.path.join(dspl_path, name))])
    number_radials = len([name for name in os.listdir(dsplRadial_path) if os.path.isfile(os.path.join(dsplRadial_path, name))])
    
    # save all samples in matrix
    samples = [] 
    for i, filename in enumerate(os.listdir(dspl_path)):
        f = os.path.join(dspl_path, filename)
        if os.path.isfile(f):
            sample = loadmat(f)
            if '__header__' in sample: del sample['__header__']
            if '__version__' in sample: del sample['__version__']
            if '__globals__' in sample: del sample['__globals__']
            sample['name'] = filename
            samples = np.append(samples, sample)
        else:
            continue
    samples = np.array(samples)

    # save all radial patterns of displacements in matrix
    dspl_radials = []
    for i, filename in enumerate(os.listdir(dsplRadial_path)):
        f = os.path.join(dsplRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            dspl_radials = np.append(dspl_radials, radial)
        else:
            continue
    dspl_radials = np.array(dspl_radials)
    
    # save all targets in matrix
    targets = []
    for i, filename in enumerate(os.listdir(trac_path)):
        f = os.path.join(trac_path, filename)
        if os.path.isfile(f):
            target = loadmat(f)
            if '__header__' in target: del target['__header__']
            if '__version__' in target: del target['__version__']
            if '__globals__' in target: del target['__globals__']
            target['name'] = filename
            targets = np.append(targets, target)
        else:
            continue 
    targets = np.array(targets)
    
    # save all radial patterns of traction forces in matrix
    trac_radials = []
    for i, filename in enumerate(os.listdir(tracRadial_path)):
        f = os.path.join(tracRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            trac_radials = np.append(trac_radials, radial)
        else:
            continue
    trac_radials = np.array(trac_radials)

    return samples, dspl_radials, targets, trac_radials

Create arrays containing 'brdx', 'brdy', 'dspl' or 'trac' and 'name'

In [6]:
samples, dspl_radials, targets, trac_radials = data_to_npArrays(dspl_path, dsplRadial_path, trac_path, tracRadial_path)
samples, targets = np.append(samples, dspl_radials), np.append(targets, trac_radials)

Create arrays for training and validation and specify batch size

In [7]:
# Currently, X and y are of shape (samples, width, height, depth)
X, y = np.array([sample['dspl'] for sample in samples]), np.array([target['trac'] for target in targets])
# Reshape to (samples, channels, depth, height, width)
X = np.moveaxis(X[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
y = np.moveaxis(y[:, np.newaxis], [2, 3, 4], [-1, 3, 2])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
batch_size = 32

Create a `DataSet` and `DataLoader`

In [35]:
train_set = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
val_set = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))

dataloaders = {}
dataloaders['train'] = DataLoader(train_set, batch_size=batch_size, shuffle=True)
dataloaders['val'] = DataLoader(val_set, batch_size=2 * batch_size)

## Visualize data

Plot displacement of given sample

In [36]:
def plotDspl(sample):
    zipped = np.array(list(zip(sample['brdx'][0], sample['brdy'][0])))  # array with (x,y) pairs of cell border coordinates
    polygon = sh.geometry.Polygon(zipped)  # create polygon

    interior = np.zeros((sample['dspl'].shape[0], sample['dspl'].shape[1]), dtype=int)  # create all zero matrix
    for i in range(len(interior)):  # set all elements in interior matrix to 1 that actually lie within the cell
        for j in range(len(interior[i])):
            point = Point(i, j)
            if polygon.contains(point):
                interior[i][j] = 1

    # plot polygon using geopandas
    p = gpd.GeoSeries(polygon)
    p.plot()
    plt.show()

tbd: Plot traction field of given target

In [37]:
def plotTrac(target):
    return None

## Build model

Specify model name and logs for model evaluation

In [38]:
NAME = "TracNet104-{}".format(int(time.time()))
# tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [39]:
class ConvBlock_1(nn.Module):
    """Conv3D -> BatchNorm -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=(2,3,3), padding='same', bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_block_1(x)

In [40]:
class ConvBlock_2(nn.Module):
    """Conv3D -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_2 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=(2,3,3), padding='same', bias=False),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_block_2(x)

In [41]:
class Concat(nn.Module):
    """Concatenate two tensors"""
    
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def forward(self, x, y):
        return torch.cat(x, y, dim=1)

In [61]:
class TracNet(nn.Module):
    def __init__(self, n_channels):
        super().__init__()
        
        self.s1 = ConvBlock_1(n_channels, 32)
        self.s2 = ConvBlock_2(32, 64)
        self.s3 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s4 = ConvBlock_1(64, 64)
        self.s5 = ConvBlock_2(64, 128)
        self.s6 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s7 = ConvBlock_1(128, 128)
        self.s8 = ConvBlock_2(128, 256)
        self.s9 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s10 = ConvBlock_1(256, 128)
        self.s11 = ConvBlock_1(128, 256)
        self.s12 = nn.ConvTranspose3d(256, 256, kernel_size=(2,3,3), bias=False)
        #fusion3
        self.s13 = ConvBlock_1(512, 64)
        self.s14 = ConvBlock_1(64, 128)
        self.s15 = nn.ConvTranspose3d(128, 128, kernel_size=(2,3,3), bias=False)
        #fusion2
        self.s16 = ConvBlock_1(256, 32)
        self.s17 = ConvBlock_1(32, 64)
        self.s18 = nn.ConvTranspose3d(64, 64, kernel_size=(2,3,3), bias=False)
        #fusion1
        self.s19 = ConvBlock_1(128, 1)
        self.s20 = ConvBlock_1(1, 32)
        self.s21 = nn.Conv3d(32, 1, kernel_size=(2,3,3), bias=False)
        
    def forward(self, x):
        x1 = self.s1(x)
        x2 = self.s2(x1)
        x3 = self.s3(x2)
        x4 = self.s4(x3)
        x5 = self.s5(x4)
        x6 = self.s6(x5)
        x7 = self.s7(x6)
        x8 = self.s8(x7)
        x9 = self.s9(x8)
        x10 = self.s10(x9)
        x11 = self.s11(x10)
        x12 = self.s12(x11)
        fusion3 = Concat(x8, x12)
        x13 = self.s13(fusion3)
        x14 = self.s14(x13)
        x15 = self.s15(x14)
        fusion2 = Concat(x5, x15)
        x16 = self.s16(fusion2)
        x17 = self.s17(x16)
        x18 = self.s18(x17)
        fusion1 = Concat(x2, x18)
        x19 = self.s19(fusion1)
        x20 = self.s20(x19)
        logits = self.s21(x20)
        return logits


In [62]:
nn.ConvTranspose3d?

In [63]:
def initialize_weight(module):
    if isinstance(module, (nn.Conv3d, nn.ConvTranspose3d)):
        nn.init.xavier_normal_(module.weight)

In [64]:
def fit(model, optimizer, dataloaders, num_epochs):
    for epoch in range(num_epochs):
        # Training
        model.train()
        for xb, yb in dataloaders["train"]:
            logits = model(xb)
            loss = nn.MSELoss(logits, yb)
            model.zero_grad()
            loss.backward()
            optimizer.step
            
        # Evaluation
        model.eval()
        loss = 0
        with torch.no_grad():
            for xb, yb in dataloaders["val"]:
                logits = model(xb)
                loss += nn.MSELoss(logits, yb).sum()
                avg_loss = loss / len(dataloaders["val"])
                
        print(f"Epoch {epoch}: val_loss = {loss:.3f}")   

In [66]:
model = TracNet(n_channels=1)
model.apply(initialize_weight)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

fit(model, optimizer, dataloaders, num_epochs=2)

TypeError: conv3d() received an invalid combination of arguments - got (Concat, Parameter, NoneType, tuple, str, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!Concat!, !Parameter!, !NoneType!, !tuple!, !str!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!Concat!, !Parameter!, !NoneType!, !tuple!, str, !tuple!, int)
